# Subduction plots

In [ ]:
import os
import numpy
from IPython.display import Image
from IPython.core.display import HTML 
from openquake.sub.misc.profile import _read_profiles
from openquake.sub.misc.edge import create_from_profiles
from openquake.sub.misc.utils import create_inslab_meshes
from openquake.sub.quad.msh import create_lower_surface_mesh

In [ ]:
tmp_path = "./tmp"
if not os.path.exists(tmp_path):
    os.mkdir(tmp_path)
    os.mkdir(os.path.join(tmp_path, 'vf'))    

In [ ]:
path = "./data/"
profiles, _ = _read_profiles(path)
slab_thickness = 50.
sampling = 40
idl = False
alg = False
smsh = create_from_profiles(profiles, sampling, sampling, idl, alg)
low_mesh = create_lower_surface_mesh(smsh, 50)

## Plotting profiles

In [ ]:
%%bash -s "$path" 

catalogue='./data/model_catalogue_subcatalogues_cat1_zon2_cat1_TR_slab_6.csv'

gmt set MAP_FRAME_TYPE = PLAIN
gmt set PS_MEDIA = a4

ext='-R-82/-73/-7/0/0/100'
prj='-Jm1 -Jz-0.01 -p80/10/0'
out='./tmp/profiles.ps'

DATA=$1'cs*'

gmt psbasemap $ext $prj -K -B+n > $out
# tail -n+2 $catalogue | gawk -F, '{print $10, $11, $15}' | gmt psxyz $ext $prj -O -K -Su0.1 >> $out

for f in $DATA; 
do 
    gmt psxyz $f $ext $prj -O -K -Wthinner,green,- >> $out
    gmt psxy $f $ext $prj -O -K -Wred >> $out
done

gmt psbasemap $ext $prj -O -B+n >> $out
gmt psconvert $out -A0.5c -P -Tg 
echo 'done'

In [ ]:
Image(url= "./tmp/profiles.png")

## Plotting mesh

First we create a temporary file containing the edges created starting from the profiles.

In [ ]:
def write_mesh(smsh, key):
    edges_fname = './tmp/{:s}_edges.xyz'.format(key)
    fou = open(edges_fname, 'w')
    fou.write('>\n')
    for e in smsh:
        for i in numpy.nonzero(numpy.isfinite(e[:,2]))[0]:
            fou.write('{:.5f},{:.5f},{:.5f}\n'.format(e[i,0], e[i,1], e[i,2]))
        fou.write('>\n')
    fou.close()

    profiles_fname = './tmp/{:s}_profiles.xyz'.format(key)
    fou = open(profiles_fname, 'w')
    fou.write('>\n')
    last = -1
    cont = False
    for i in range(len(smsh[0])):
        fou.write('>\n')
        for e in smsh:   
            if numpy.all(numpy.isfinite(e[i])):
                fou.write('{:.5f},{:.5f},{:.5f}\n'.format(e[i,0], e[i,1], e[i,2]))
    fou.write('>\n')
    fou.close()
    return edges_fname, profiles_fname

def write_poly(smsh, key):
    poly_fname = './tmp/{:s}_poly.xyz'.format(key)
    fou = open(poly_fname, 'w')
    fou.write('>\n')
    for i1 in range(len(smsh)-1):
        for i2 in range(len(smsh[0])-1):
            if (numpy.all(numpy.isfinite(smsh[i1][i2,:])) and 
                numpy.all(numpy.isfinite(smsh[i1][i2+1,:])) and
                numpy.all(numpy.isfinite(smsh[i1+1][i2,:])) and
                numpy.all(numpy.isfinite(smsh[i1+1][i2+1,:]))):
                e = smsh[i1]
                f = smsh[i1+1]
                fou.write('{:.5f},{:.5f},{:.5f}\n'.format(e[i2,0], e[i2,1], e[i2,2]))
                fou.write('{:.5f},{:.5f},{:.5f}\n'.format(f[i2,0], f[i2,1], f[i2,2]))
                fou.write('{:.5f},{:.5f},{:.5f}\n'.format(f[i2+1,0], f[i2+1,1], f[i2+1,2]))
                fou.write('{:.5f},{:.5f},{:.5f}\n'.format(e[i2+1,0], e[i2+1,1], e[i2+1,2]))
                fou.write('{:.5f},{:.5f},{:.5f}\n'.format(e[i2,0], e[i2,1], e[i2,2]))
                fou.write('>\n')
    fou.close()
    return poly_fname

In [ ]:
edges_fname, profiles_fname = write_mesh(smsh, 'top')
edges_low_fname, profiles_low_fname = write_mesh(low_mesh, 'low')
poly_fname = write_poly(smsh, 'poly')
poly_low_fname = write_poly(low_mesh, 'poly_low')

In [ ]:
%%bash -s "$path" "$edges_fname" "$profiles_fname" "$poly_low_fname" "$poly_fname"

ext='-R-82/-75/-7/0/0/100'
prj='-Jm1 -Jz-0.01 -p80/10/0'
out='./tmp/edges.ps'
DATA=$1'cs*'

gmt psbasemap $ext $prj -K -B+b > $out

gmt psxyz $4 $ext $prj -O -K -Wdefault,orange -G240 >> $out
gmt psxyz $5 $ext $prj -O -K -Wthinnest,purple -G220 >> $out

for f in $DATA; 
do 
    gmt psxyz $f $ext $prj -O -K -Wthinner,green,- >> $out
done

gmt psbasemap $ext $prj -O -B+b >> $out
gmt psconvert $out  -A0.5c -P -Tg 
echo 'done'

In [ ]:
Image(url= "./tmp/edges.png")

## Plotting virtual faults

In [ ]:
dips = [45, 135]
sampling = 20.
ohs = create_inslab_meshes(smsh, dips, slab_thickness, sampling)

In [ ]:
for key in ohs:
    for i, fault in enumerate(ohs[key]):
        fou = open(os.path.join(tmp_path, 'vf', 'vf_{:d}_{:03d}.txt'.format(key, i)), 'w')
        for pro in fault:
            fou.write('>\n')
            for p in pro:
                fou.write('{:.5f},{:.5f},{:.5f}\n'.format(p.longitude, p.latitude, p.depth))
        fou.write('>\n')
        fou.close()

In [ ]:
sampling = 15.
tmp_mesh = create_from_profiles(ohs[135][4], sampling, sampling, idl, alg)
vf_edges_fname, vf_profiles_fname = write_mesh(tmp_mesh, 'vf')
vf_poly_fname = write_poly(tmp_mesh, 'vf_poly')

In [ ]:
%%bash -s "$path" "$vf_edges_fname"  "$vf_profiles_fname" "$poly_low_fname"  "$poly_fname" "$vf_poly_fname"
#          1       2                 3                    4                  5                     

ext='-R-82/-73/-7/0/0/150'
prj='-Jm1 -Jz-0.01 -p80/10/0'
out='./tmp/virtual_faults0.ps'
DATA=$1'cs*'

gmt psbasemap $ext $prj -K -B+b > $out

gmt psxyz $4 $ext $prj -O -K -Wdefault,orange -G240 >> $out
gmt psxyz $5 $ext $prj -O -K -Wthinnest,purple -G220 >> $out
gmt psxyz $6 $ext $prj -O -K -Wthinnest,red -G200 >> $out
gmt psxyz $4 $ext $prj -O -K -Wdefault,orange,. >> $out
gmt psxyz $5 $ext $prj -O -K -Wthinnest,purple >> $out

#gmt psxyz ./tmp/vf/vf_45_004.txt $ext $prj -O -K -Wthinner,blue >> $out
gmt psxyz ./tmp/vf/vf_135_004.txt $ext $prj -O -K -Wthinner,red >> $out

#gmt psxyz $4 $ext $prj -O -K -Wthinnest,blue >> $out
#gmt psxyz $5 $ext $prj -O -K -Wthinnest,blue >> $out

#for f in $DATA; 
#do 
#    gmt psxyz $f $ext $prj -O -K -Wthinner,green,- >> $out
#done

gmt psbasemap $ext $prj -O -B+n >> $out
gmt psconvert $out -A0.5c -P -Tg 
echo 'done'

In [ ]:
Image(url= "./tmp/virtual_faults0.png")

In [ ]:
%%bash -s "$path" "$vf_edges_fname"  "$vf_profiles_fname" "$poly_low_fname"  "$poly_fname" "$vf_poly_fname"
#          1       2                 3                    4                  5                     

ext='-R-82/-73/-7/0/0/150'
prj='-Jm1 -Jz-0.01 -p3200/10/0'
out='./tmp/virtual_faults1.ps'
DATA=$1'cs*'

gmt psbasemap $ext $prj -K -B+b > $out

gmt psxyz $4 $ext $prj -O -K -Wdefault,orange -G240 >> $out
gmt psxyz $5 $ext $prj -O -K -Wthinnest,purple -G220 >> $out
gmt psxyz $6 $ext $prj -O -K -Wthinnest,red -G200 >> $out
gmt psxyz $5 $ext $prj -O -K -Wthinnest,purple >> $out

#gmt psxyz ./tmp/vf/vf_45_004.txt $ext $prj -O -K -Wthinner,blue >> $out
gmt psxyz ./tmp/vf/vf_135_004.txt $ext $prj -O -K -Wthinner,red >> $out

#gmt psxyz $4 $ext $prj -O -K -Wthinnest,blue >> $out
#gmt psxyz $5 $ext $prj -O -K -Wthinnest,blue >> $out

#for f in $DATA; 
#do 
#    gmt psxyz $f $ext $prj -O -K -Wthinner,green,- >> $out
#done

gmt psbasemap $ext $prj -O -B+n >> $out
gmt psconvert $out -A0.5c -P -Tg 
echo 'done'

In [ ]:
I23mage(url= "./tmp/virtual_faults1.png")